In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as optimize
from iminuit import Minuit

In [2]:
def chisq(obs, exp, error):
    return np.sum(np.square(obs - exp) / np.square(error))

We will first try with the usual broken power law:

In [3]:
def br_po_lw(xdata, E1, fl_0, lam1, sqrtdelta_lam12):
    E0 = xdata[0]
    polw = np.zeros(len(xdata))
    lam2 = lam1 + sqrtdelta_lam12**2 #for multiple broken power law add lam_i = lam_(i-1) + delta_lam
    phi_0 = fl_0
    phi_1 = phi_0 * ((E1/E0) ** (-lam1))
    for i in range(len(xdata)):
        if xdata[i] < E1:#you can add E>E0
            polw[i] = phi_0 * ((xdata[i]/E0) ** (-lam1))
        else:#you can add elifs for multiple broken power law
            polw[i] = phi_1 * ((xdata[i]/E1) ** (-lam2))
    return polw

Now let's add the $\chi^2$ to the function in order to minimize:

In [4]:
def br_po_lw_chsq(params): 
    #you need to define theE, SED and SED_u before(E and SED must be a numpy.array and SED an scalar or a numpy.array)
    E1, fl_0, lam1, sqrtdelta_lam12 = params
    E0 = xdata[0]
    polw = np.zeros(len(xdata))
    lam2 = lam1 + sqrtdelta_lam12**2 #for multiple broken power law add lam_i = lam_(i-1) + delta_lam
    phi_0 = fl_0
    phi_1 = phi_0 * ((E1/E0) ** (-lam1))
    for i in range(len(xdata)):
        if xdata[i] < E1:#you can add E>E0
            polw[i] = phi_0 * ((xdata[i]/E0) ** (-lam1))
        else:#you can add elifs for multiple broken power law
            polw[i] = phi_1 * ((xdata[i]/E1) ** (-lam2))
    return np.sum(np.square(ydata - polw) / np.square(ydata_u))

In this case we will define the multiple broken power law, but as it depends on the number of knots it will be much longer.

In [6]:
#m = Minuit(LSQ, a=5, b=5,
 #          error_a=0.1, error_b=0.1,
 #          limit_a=(0, None), limit_b=(0, 10),
 #          errordef=1)

from now on the position of the knots will not be a free parameter anymore in order to make the convergence of the fit easier

In [8]:
def multiple_bpl(params):
    print('Warning: the shape of the parametes will depend on the number of knots (defined as knots)')
    if knots < 3 or knots >3: #change this when adding more number of knots
        raise Exception('at thins moment knots can only be equal to 3')
    polw = np.zeros(len(xdata))
    E0 = xdata[0]
    if knots == 3:
        phi_0, lam0, sqrtdelta_lam01 = params
        E1 = (E0 + xdata[-1]) / 2
        lam1 = lam0 + sqrtdelta_lam01**2 
        phi_1 = phi_0 * ((E1/E0) ** (-lam0))
        for i in range(len(xdata)):
            if xdata[i] < E1:
                polw[i] = phi_0 * ((xdata[i]/E0) ** (-lam0))
            else:
                polw[i] = phi_1 * ((xdata[i]/E1) ** (-lam1))







    return polw


In [9]:
# least-squares score function = sum of data residuals squared
def LSQ(params):
    return np.sum((ydata - multiple_bpl(xdata, params)) ** 2 / ydata_u ** 2)

In [19]:
def multiple_bpl_fit(knots, xdata, ydata, ydata_u, initial_guess):
    LSQ.errordef = Minuit.LIKELIHOOD

    m = Minuit(LSQ, initial_guess)

    m.migrad()
    print(m.values)

    m.hesse()
    print(m.errors)

    return m